---
모델 구조 확인

In [1]:
import torch
from torch import nn


class CustomModel(nn.Module):
    pass

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("../models/model.pt", map_location=device)
print(model)

CustomModel(
  (layer): Linear(in_features=2, out_features=1, bias=True)
)


---
모델 상태 저장

In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [2]:

train_dataset = CustomDataset("../datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [3]:

for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[ 3.0970, -1.7006]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.6710], device='cuda:0', requires_grad=True)], Cost : 0.092
Epoch : 2000, Model : [Parameter containing:
tensor([[ 3.0975, -1.7031]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.6560], device='cuda:0', requires_grad=True)], Cost : 0.081
Epoch : 3000, Model : [Parameter containing:
tensor([[ 3.0976, -1.7036]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.6423], device='cuda:0', requires_grad=True)], Cost : 0.085
Epoch : 4000, Model : [Parameter containing:
tensor([[ 3.0976, -1.7034]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.6296], device='cuda:0', requires_grad=True)], Cost : 0.077
Epoch : 5000, Model : [Parameter containing:
tensor([[ 3.0979, -1.7034]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.6181], device='cuda:0', requires_grad=True)]

In [4]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.9696],
        [ 69.5241],
        [356.7714]], device='cuda:0')


In [5]:
torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)


---
모델 상태 불러오기

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)

model_state_dict = torch.load("../models/model_state_dict.pt", map_location=device)
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [ ]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
print(output)